# OpenAI 외 도구 호출 에이전트(Tool Calling Agent)

OpenAI 외에도 `Anthropic`, `Google Gemini`, `Together.ai`, `Ollama`, `Mistral`과 같은 더 광범위한 공급자 구현을 지원합니다.

이번 챕터에서는 다양한 LLM 을 사용하여 도구 호출 에이전트를 생성하고 실행하는 방법을 살펴보겠습니다.

**참고 링크**

- [LangChain 공식 도큐먼트](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/)

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Agents")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agents


In [3]:
from langchain.tools import tool
from typing import List, Dict
from langchain_teddynote.tools import GoogleNews


# 도구 정의
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)


print(f"도구 이름: {search_news.name}")
print(f"도구 설명: {search_news.description}")

도구 이름: search_news
도구 설명: Search Google News by input keyword


In [4]:
# tools 정의
tools = [search_news]

## Agent 용 프롬프트 생성

- `chat_history` : 이전 대화 내용을 저장하는 변수 (멀티턴을 지원하지 않는다면, 생략 가능합니다.)
- `agent_scratchpad` : 에이전트가 임시로 저장하는 변수
- `input` : 사용자의 입력

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `search_news` tool for searching keyword related news.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

## Tool Calling 을 지원하는 다양한 LLM 목록

실습 진행을 위해서는 아래 내용을 설정해야 합니다.

**Anthropic**

- [Anthropic API 키 발급 관련](https://console.anthropic.com/settings/keys)
- `.env` 파일 내 `ANTHROPIC_API_KEY` 에 발급받은 키를 설정하세요

**Gemini**

- [Gemini API 키 발급 관련](https://aistudio.google.com/app/apikey?hl=ko)
- `.env` 파일 내 `GOOGLE_API_KEY` 에 발급받은 키를 설정하세요

**Together AI**

- [Together AI API 키 발급 관련](https://api.together.ai/)
- `.env` 파일 내 `TOGETHER_API_KEY` 에 발급받은 키를 설정하세요

**Ollama**

- [Ollama Tool Calling 지원 모델 리스트](https://ollama.com/search?c=tools)
- [이번 실습에 사용할 llama3.1 모델](https://ollama.com/library/llama3.1)
- 터미널 창에 `ollama pull llama3.1` 명령어를 입력하여 모델을 다운로드 받습니다.
- 이전에 Ollama 를 사용하지 않았다면, [Ollama](https://wikidocs.net/233805) 를 참고해 주세요.

langchain-ollama 설치를 한 뒤 진행해 주세요.

In [6]:
# !pip install -qU langchain-ollama==0.1.3

In [26]:
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
import os

# GPT-4o-mini
gpt = ChatOpenAI(model="gpt-4o-mini")

# Claude-3-5-sonnet
claude = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)

# Gemini-1.5-pro-latest
gemini = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)

# Llama-3.1-70B-Instruct-Turbo
llama = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
)

# Llama-3.1
ollama = ChatOllama(model="llama3.1", temperature=0)

LLM 기반으로 Agent 를 생성합니다.

In [27]:
# Agent 생성
gpt_agent = create_tool_calling_agent(gpt, tools, prompt)
claude_agent = create_tool_calling_agent(claude, tools, prompt)
gemini_agent = create_tool_calling_agent(gemini, tools, prompt)
llama_agent = create_tool_calling_agent(llama, tools, prompt)
ollama_agent = create_tool_calling_agent(ollama, tools, prompt)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


## AgentExecutor 생성 후 실행 및 결과 확인



In [11]:
from langchain.agents import AgentExecutor

# gpt_agent 실행
agent_executor = AgentExecutor(
    agent=gpt_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

result = agent_executor.invoke({"input": "AI 투자와 관련된 뉴스를 검색해 주세요."})

print("Agent 실행 결과:")
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `search_news` with `{'query': 'AI 투자'}`


[{'url': 'https://news.google.com/rss/articles/CBMiTkFVX3lxTE5xUEdrNHZZclQ4RUNmNGpqZG1COUNfckVOZ2hqQTJ4NGkwa0NRU0Q1azRQNzMwTTBJVDloQmNKamRZc3dic0plQWtEUTNXUQ?oc=5', 'content': 'AI 투자 늘리는 통신사, 주파수 재할당 대가 마련 부담 - 전자신문'}, {'url': 'https://news.google.com/rss/articles/CBMiU0FVX3lxTFB1VDRjTlIxeHRjV3NKaERHMFJLOFNpUlNUVnZzQ0JvOUpISVE2cW9rbTZPVU9lQXZuUURJNHF0aDZyaDdQM0F5NFhsc0NrN1dHNEZz?oc=5', 'content': '글로벌 CEO들 "향후 3년 무조건 AI 투자···직원도 더 뽑을 것" - 네이트 뉴스'}, {'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTE9BQUtjd3pDS1dXRkZQTlo4Y2w2d2laZFFQRXdsbGlCMFRTcWJYLTNLaksxbFlnYzk3clVnYlFxZF93U2xubXY1VVhBTDQ2SDhLbm9hcDdXQ3prTlJtV0VCYTFINmNRbjR5N3c?oc=5', 'content': 'MIT 경제학자 "10년간 AI에 영향받을 직업은 5% 불과...기업은 투자비만 날리게 될 것" - AI타임스'}, {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE5XeGtRRjRyeUFZZng4U2JWMmhZdFczSmR5d0YwbmF3cHpJVjRka2x4ZmRoVjdnaWdLOUdVSjNtN1FqeTBpTmRaUk9IZFFVamhGREZzQloxVmZ3UdIBVE

다양한 llm을 사용하여 에이전트를 실행합니다.

다음은 입력받은 llm을 사용하여 Agent 를 생성하고 실행하여 결과를 출력하는 함수입니다.

In [16]:
def execute_agent(llm, tools, input_text, label):
    agent = create_tool_calling_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
    result = executor.invoke({"input": input_text})
    print(f"[{label}] 결과입니다.")
    print(result["output"])

각 llm 별로 에이전트를 생성하고 실행하여 결과를 출력합니다.

In [17]:
query = (
    "AI 투자와 관련된 뉴스를 검색하고, 결과를 Instagram 게시글 형식으로 작성해 주세요."
)

In [18]:
# gpt
execute_agent(gpt, tools, query, "gpt")

[gpt] 결과입니다.
🌟 **AI 투자 소식!** 🌟

1️⃣ **통신사들의 AI 투자** 📡  
통신사들이 AI 투자를 늘리고 있습니다. 하지만 주파수 재할당에 따른 부담이 크다고 하는데요. 더 자세한 내용은 [여기에서 확인하세요](https://news.google.com/rss/articles/CBMiTkFVX3lxTE5xUEdrNHZZclQ4RUNmNGpqZG1COUNfckVOZ2hqQTJ4NGkwa0NRU0Q1azRQNzMwTTBJVDloQmNKamRZc3dic0plQWtEUTNXUQ)! 

2️⃣ **CEO들의 의견** 👔  
글로벌 CEO들이 "향후 3년간 무조건 AI에 투자할 것"이라고 발표했습니다. 직원도 더 뽑을 예정이라고 하네요! 자세한 내용은 [여기에서 확인하세요](https://news.google.com/rss/articles/CBMiU0FVX3lxTFB1VDRjTlIxeHRjV3NKaERHMFJLOFNpUlNUVnZzQ0JvOUpISVE2cW9rbTZPVU9lQXZuUURJNHF0aDZyaDdQM0F5NFhsc0NrN1dHNEZz)! 

3️⃣ **MIT 경제학자의 경고** ⚠️  
MIT의 경제학자가 "앞으로 10년간 AI에 영향을 받을 직업은 5% 불과하다"며 기업들이 투자비를 날릴 가능성이 있다고 경고했습니다. 더 알아보려면 [여기에서 확인하세요](https://news.google.com/rss/articles/CBMiakFVX3lxTE9BQUtjd3pDS1dXRkZQTlo4Y2w2d2laZFFQRXdsbGlCMFRTcWJYLTNLaksxbFlnYzk3clVnYlFxZF93U2xubXY1VVhBTDQ2SDhLbm9hcDdXQ3prTlJtV0VCYTFINmNRbjR5N3c)! 

4️⃣ **시애틀의 의료 AI 투자** 🏥  
아마존과 MS가 수조 원을 투자해 시애틀이 의료 AI의 메카로 떠오르고 있습니다. 자세한 내용은 [여기에서 확인하세요](https://news.google.com/rss/articles/CBMiWkFVX3

In [28]:
# claude
execute_agent(claude, tools, query, "claude")

[claude] 결과입니다.
[{'text': '\n\n검색 결과를 바탕으로 Instagram 게시글 형식으로 작성해 드리겠습니다.\n\n---\n\n📱 #AI투자 #테크트렌드 #미래기술\n\n🔍 AI 투자 핫이슈 모음! 🤖💰\n\n1. 통신사들의 AI 투자 확대 🚀\n   - 주파수 재할당 대가 마련 부담에도 불구하고 AI 투자 증가\n   - 미래 경쟁력 확보를 위한 전략적 움직임\n\n2. 글로벌 CEO들의 AI 투자 전망 📈\n   - 향후 3년간 AI 투자 필수\n   - 인력 채용 확대 예상\n   - AI 시대에 대비한 기업 전략 수립 중\n\n3. MIT 경제학자의 AI 영향 분석 🧠\n   - 10년간 AI 영향 받는 직업은 5% 정도\n   - 과도한 투자에 대한 경고\n   - 신중한 AI 도입 및 투자 필요성 제기\n\n4. 의료 AI의 메카, 시애틀 🏥\n   - 아마존, MS 등 대기업의 수조원 투자\n   - 의료 분야 AI 혁신의 중심지로 부상\n   - 헬스케어와 AI의 융합 가속화\n\n5. AI 스타트업 투자 소식 💼\n   - 마크비전, 220억원 규모 시리즈A+ 투자 유치 성공\n   - AI 기업에 대한 투자자들의 높은 관심 반영\n\n💡 AI 기술은 다양한 산업 분야에 혁신을 가져오고 있습니다. 하지만 신중한 접근과 전략적 투자가 필요한 시점이에요. 여러분은 AI 시대에 어떻게 준비하고 계신가요? 댓글로 여러분의 생각을 공유해주세요! 👇\n\n#AI혁명 #투자전략 #기술혁신 #미래일자리 #의료AI\n\n---\n\n이상으로 AI 투자 관련 뉴스를 Instagram 게시글 형식으로 작성해 보았습니다. 이 게시글은 주요 뉴스 내용을 요약하고, 해시태그와 이모지를 사용하여 Instagram 스타일로 구성했습니다. 또한, 독자들의 참여를 유도하는 질문으로 마무리하여 engagement를 높이고자 했습니다.', 'type': 'text', 'index': 0}]


In [20]:
# gemini
execute_agent(gemini, tools, query, "gemini")

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


[gemini] 결과입니다.
## 🔥 핫한 AI 투자 소식! 놓치지 마세요! 🔥

🤖 요즘 AI 투자가 뜨겁다는데, 무슨 일이 벌어지고 있는지 궁금하시죠? 🤔 

📰 주요 뉴스 헤드라인만 후다닥 살펴보세요!

* **AI 투자 늘리는 통신사, 주파수 재할당 대가 마련 부담** - 전자신문
* **글로벌 CEO들 "향후 3년 무조건 AI 투자···직원도 더 뽑을 것"** - 네이트 뉴스
* **MIT 경제학자 "10년간 AI에 영향받을 직업은 5% 불과...기업은 투자비만 날리게 될 것"** - AI타임스
* **아마존·MS가 수조원 투자…의료AI 메카 된 시애틀** - 한국경제
* **인공지능 기업 ‘마크비전’, 220억원 규모 시리즈A+ 투자 유치** - 플래텀

#AI #투자 #인공지능 #뉴스 #놓치지마세요 



In [24]:
# llama3.1 70B (Together.ai)
execute_agent(
    llama,
    tools,
    "Search AI related news and write it in Instagram post format",
    "llama3.1 70B",
)

[llama3.1 70B] 결과입니다.
Here's an Instagram post format for AI related news:

"Stay up-to-date with the latest AI news!

Meta's new AI-powered video generation tool, "MovieGen", has been released! 

The SK AI Summit is happening in Seoul, where experts and entrepreneurs will gather to discuss the future of AI and how to coexist with AGI.

A recent hurricane damage photo that went viral was actually created by AI!

Meta's AR technology is giving Apple a run for its money!

AI talent tech company, Glorand, has launched its AI diagnosis SaaS 'PSYT' in Japan.

Stay informed and join the conversation! #AI #ArtificialIntelligence #TechNews"

Sources:
- https://news.google.com/rss/articles/CBMiakFVX3lxTFB2MmRiRVZWWHlGNnhfTDFfRldrYllYSVBlNi1NRTBiN2xYMGc2NHhHY3JsUkF5dDR1aXdwRDBmZXI4QzUwNGQ0aU5NOG1ZUVQ1Y2MwQy1vS3dXTVQxbV9FanRqS1FDLW5RaGc?oc=5
- https://news.google.com/rss/articles/CBMiSEFVX3lxTE16M19mWXBHTnRPSlhCNkZzVmlSV0N1QjZKbXNlcHN5QWxVYVRUd3RaeTFSZkpleG40ZWRubWNTSzFaS0o1eGM4cQ?oc=5
- https://

In [22]:
# llama3.1 8B (ollama)
execute_agent(ollama, tools, query, "llama3.1(Ollama)")

[llama3.1(Ollama)] 결과입니다.
**AI 투자 뉴스**

* Bridgewise가 개발한 BRIDGET는 세계 최초의 대화형 AI 투자 도구로, 규제 준수한 투자 추천을 제공합니다.
* 미디훼일은 시리즈 B2 펀딩 15.8억 원을 확보하여 총 투자금액 30.5억 원을 넘어섰습니다.
* GAIIP는 대규모 AI 인프라 투자를 위한 글로벌 프로젝트를 시작했습니다.
* 플레이태그는 AI 행동 분석 산업에 도전장을 내걸었습니다.

**Instagram 게시글**

"AI 투자 뉴스!

Bridgewise의 BRIDGET, 세계 최초의 대화형 AI 투자 도구!
미디훼일, 시리즈 B2 펀딩 15.8억 원 확보!
GAIIP, 대규모 AI 인프라 투자 프로젝트 시작!
플레이태그, AI 행동 분석 산업에 도전장 내걸기!

#AInvestment #Bridgewise #BRIDGET #Mediwhale #GAIIP #Playtag"
